<a href="https://colab.research.google.com/github/zahrabashir98/Face-Recognition-and-Tracking/blob/master/CVFinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [0]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 31kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=1668d89cfd0b74e1276c45916d905607dc141cc9b5a0b63d04b717cbe50755ad
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os
import imutils




print("start learning")
imagePaths = list(paths.list_images("drive/My Drive/Vision/new"))


knownEncodings = []
knownNames = []

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
	# extract the person name from the image path
	print("Processing image {}/{}".format(i + 1,len(imagePaths)))
	name = imagePath.split(os.path.sep)[4][2:-4]
	print(name)

	image = cv2.imread(imagePath)
	rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

	
	boxes = face_recognition.face_locations(rgb,model="cnn")

	encodings = face_recognition.face_encodings(rgb,known_face_locations=boxes)

	for encoding in encodings:

		knownEncodings.append(encoding)
		knownNames.append(name)
	
print(knownNames)

print("encodings...")
data = {"encodings": knownEncodings, "names": knownNames}
f = open("drive/My Drive/Vision/encodings.pickle", "wb")
f.write(pickle.dumps(data))
f.close()

start learning
Processing image 1/43
AlirezaRezaei
Processing image 2/43
AliSafarpoor
Processing image 3/43
DanialKamali
Processing image 4/43
FatemeTavakoli
Processing image 5/43
HamedKalantari
Processing image 6/43
MahsaMeymari
Processing image 7/43
OmidGhahroodi
Processing image 8/43
Ostad
Processing image 9/43
ShayanKhorsandi
Processing image 10/43
TohidAbedini
Processing image 11/43
ZahraBashir


RuntimeError: ignored

In [0]:
# convert a list of frames to video

def convert_to_video(img_array):
  height, width, layers = img_array[0].shape
  size = (width,height)
  out = cv2.VideoWriter('drive/My Drive/Vision/project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

  for i in range(len(img_array)):
      out.write(img_array[i])
  out.release()





In [0]:
import numpy as np
def intialize_kalman(c, r, w, h):

  kalman = cv2.KalmanFilter( 4,2 ,0 )  #  state/hidden, measurement, control
  kalman.transitionMatrix = np.array([[1., 0., 1., 0.],  # a rudimentary constant speed model:
                                      [0., 1., 0., 1.],  # x_t+1 = x_t + v_t
                                      [0., 0., 1., 0.],
                                      [0., 0., 0., 1.]])
  kalman.measurementMatrix = 1. * np.eye(2, 4)  # you can tweak these to make the tracker
  kalman.processNoiseCov = 1e-5 * np.eye(4, 4)  # respond faster to change and be less smooth
  kalman.measurementNoiseCov = 1e-3 * np.eye(2, 2)
  kalman.errorCovPost = 1e-1 * np.eye(4, 4)
  kalman.statePost = np.array([c + w / 2, r + h / 2, 0, 0], dtype='float64')

  return kalman


In [0]:
counter = 0

In [0]:
import cv2
import sys
import pickle
import face_recognition
import argparse
import datetime
import math
import imutils


!ls
!cd drive
!ls

nameStatus = {}
nameLastUpdate = {}
enteringTime = {}

unknowncount = 0
now = datetime.datetime.now()

logfilename = str(now.year) + '-' + str(now.month) + '-' + str(now.day) + '_logs.txt'
print(logfilename)
logfile = open(logfilename,'w')

trackers = cv2.MultiTracker_create()

kalmanStatus = {}
first_boxes = {}
people_duration = {}
tuple_list = []
NameList=["DanialKamali", "ZahraBashir", "AliSafarpoor", "AlirezaRezaei", "FatemeTavakoli", "ShayanKhorsandi", "HamedKalantari", "OmidGhahroodi", "ElaheHoseini", "MahsaMeymari", "TohidAbedini", "NewshaEstiri", "VahidMohseni"]
present_list = []
def snap_shot(image):

    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #print("[INFO] recognizing faces...")
    
    # if counter >= 50:
    #     (success, boxes) = trackers.update(rgb)
    #     for items in boxes:
    #       tuple_list.append((int(items[0]),int(items[1]),int(items[2]),int(items[3])))

    #     boxes = tuple_list
    #     print(boxes)
    #     encodings = face_recognition.face_encodings(rgb, boxes)


    # else:
    #   boxes = face_recognition.face_locations(rgb,model="cnn")
    #   encodings = face_recognition.face_encodings(rgb, boxes)
    boxes = face_recognition.face_locations(rgb,model="cnn")
    encodings = face_recognition.face_encodings(rgb, boxes)    
 
    names = []
    
    for encoding in encodings:
        matches = face_recognition.compare_faces(data["encodings"], encoding, tolerance=0.5)
        
        name = "Unknown"
        print("ghable match")
        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
            name = max(counts, key=counts.get)
        names.append(name)
        print("bade match")

    for ((top, right, bottom, left), name) in zip(boxes, names):
        # left = x
        # top = y
        # right  = w +x
        # bottom = h + y
        box = (top, right, bottom, left)
        if not box in first_boxes:
              first_boxes[name] = box
              tracker = cv2.TrackerKCF_create()
              trackers.add(tracker, rgb, box)

        else:
          pass      

        print(name,' detected')
        if name is "Unknown":
            print("UNK")
  
        #Person Detected 

        
        # if not name in kalmanStatus:
        #     kalmanStatus[name] = intialize_kalman(left, top, right-left, bottom-top)
        #     cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)            
        #     y = top - 15 if top - 15 > 15 else top + 15
        #     cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        # else:
        #     c, r, w, h = left, top, right-left, bottom-top
        #     final = kalmanStatus[name].predict()
        #     measurement = np.array([c+w/2,r+h/2], dtype='float64')
        #     if c!=0 and w!=0 and r!=0 and h!=0:
        #         final = kalmanStatus[name].correct(measurement)
        #     cv2.rectangle(image, (int(final[0]), int(final[1])), (int(final[0])+int(final[2]), int(final[1])+int(final[3])), (0, 255, 0), 2)
        #     y = top - 15 if top - 15 > 15 else top + 15
        #     cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)




        currentTime = datetime.datetime.now()
        
        if not name in nameStatus:
            logfile.write(name+' has entered at - ' + str(currentTime.hour) + ':' + str(currentTime.minute) + ':' + str(currentTime.second) + '\n')
            print(name+' has entered')
            nameStatus[name]='IN'
            if name in NameList:
              present_list.append(name)
            nameLastUpdate[name]=currentTime 
            enteringTime[name]=currentTime #save the enreting time
            people_duration[name] = 0
        
        else:
            lastUpdateTime = nameLastUpdate[name] 
            difference = currentTime - lastUpdateTime #check difference
            if difference.seconds < 1:
                print('last_update_time_too_close : not changing status')   # if difference is too little skip
            else: # we need to add the difference
                if nameStatus[name] == 'IN':
                    timein = currentTime - nameLastUpdate[name]  # get the time between two detection
                    minutesIn = math.floor(timein.seconds /60)
                    secondsIn = timein.seconds %60
                    if minutesIn > 1:
                      logfile.write(name+' has left at - ' + str(currentTime.hour) + ':' + str(currentTime.minute) + ':' + str(currentTime.second))
                      print(name+' has left.')  
                      nameStatus[name] = 'OUT'                 
                    else:
                      people_duration[name] += secondsIn
                      nameLastUpdate[name]=currentTime
                      logfile.write('...........Duration - ' + str(minutesIn) + ' mins ' + str(secondsIn) + 'secs \n' )                  
                    
                else: #if the student went out of the class and entereded again we start recording the time
                    logfile.write(name+' has entered again at - ' + str(currentTime.hour) + ':' + str(currentTime.minute) + ':' + str(currentTime.second) + '\n')
                    print(name+' has entered.')
                    nameStatus[name]='IN'
                    enteringTime[name]=currentTime
            
            nameLastUpdate[name] = currentTime

        print(people_duration)
        print(present_list)
        #Drawing rectangle
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    #cv2.imwrite("Image_boxed_"+str(count)+".jpg", image)


# load the known faces and embeddings
print("Loading encodings...")
data = pickle.loads(open('drive/My Drive/Vision/encodings.pickle', "rb").read())
# print(data)
faceCascade = cv2.CascadeClassifier('drive/My Drive/Vision/haarcascade_frontalface_alt.xml')

print("\n\n##########################CAPTURING VIDEO TO DETECT FACES##############################\n\n")
# video_capture = cv2.VideoCapture(0)
video_capture = cv2.VideoCapture('drive/My Drive/Vision/fff.mp4')

# print(video_capture)

count =0 
framenumber=0
frames = []
counter = 0
while True:
    counter += 1
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if frame is None:
      print("oomad too not")
      break
    print(framenumber)
    framenumber+=1

  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
    )

    if not len(faces) is 0:
        #print('face found'+str(count))
        count+=1
        snap_shot(frame)
        #if count is 4:
        #    break


    # Display the resulting frame
    # from google.colab.patches import cv2_imshow
    # cv2_imshow(frame)
    frames.append(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
convert_to_video(frames)

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()


2020-1-29_logs.txt  drive  sample_data
2020-1-29_logs.txt  drive  sample_data
2020-1-29_logs.txt
Loading encodings...


##########################CAPTURING VIDEO TO DETECT FACES##############################


0
ghable match
bade match
ghable match
bade match
ghable match
bade match
ghable match
bade match
ghable match
bade match
Ostad  detected
Ostad has entered
{'Ostad': 0}
[]
HamedKalantari  detected
HamedKalantari has entered
{'Ostad': 0, 'HamedKalantari': 0}
['HamedKalantari']
TohidAbedini  detected
TohidAbedini has entered
{'Ostad': 0, 'HamedKalantari': 0, 'TohidAbedini': 0}
['HamedKalantari', 'TohidAbedini']
AlirezaRezaei  detected
AlirezaRezaei has entered
{'Ostad': 0, 'HamedKalantari': 0, 'TohidAbedini': 0, 'AlirezaRezaei': 0}
['HamedKalantari', 'TohidAbedini', 'AlirezaRezaei']
DanialKamali  detected
DanialKamali has entered
{'Ostad': 0, 'HamedKalantari': 0, 'TohidAbedini': 0, 'AlirezaRezaei': 0, 'DanialKamali': 0}
['HamedKalantari', 'TohidAbedini', 'AlirezaRezaei', 'DanialKam

In [0]:
    # if counter == 49 :
      #   print(boxes)
    # print(boxes)
    # for nameee in NameList:   
    #   if not namee in first_boxes:
    #     boxes = face_recognition.face_locations(rgb,model="cnn")
    #     encodings = face_recognition.face_encodings(rgb, boxes)
    #     first_boxes[namee] = box
        # for encoding in encodings:
        # matches = face_recognition.compare_faces(data["encodings"], encoding, tolerance=0.5)
        
        # name = "Unknown"

        # if True in matches:
        #     matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        #     # print()
        #     counts = {}
        #     for i in matchedIdxs:
        #         name = data["names"][i]
        #         counts[name] = counts.get(name, 0) + 1
        #     name = max(counts, key=counts.get)
        # names.append(name)

        # NameList.append(name????)
          
      # else:
      #   (success, boxes) = trackers.update(image)
      #   encodings = face_recognition.face_encodings(rgb, boxes)

      # encodings = face_recognition.face_encodings(rgb, boxes)
    # if 